<a href="https://colab.research.google.com/github/sainikhit2k/DL/blob/master/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from keras.layers import Input, Dense, Flatten, Conv2D, MaxPool2D
from keras.models import Model, Sequential


Using TensorFlow backend.


In [2]:
def ConvBlock(inp, n_filters, kernel_size, pad, act, name, layers):
  block = Sequential()
  for i in range(1, layers + 1):
    inp = Conv2D(n_filters, kernel_size=kernel_size, padding=pad, activation=act, name=name+'conv'+str(i))(inp)
  inp = MaxPool2D(pool_size=(2, 2), name=name+'pool')(inp)
  return inp

In [3]:
def VGG16(input_shape=(224, 224, 3)):
  input_img = Input(shape=input_shape, name='Input_Img')
  out = ConvBlock(input_img, n_filters=64, kernel_size=(3, 3), pad='same', act='relu', name='block1_', layers=2)
  out = ConvBlock(out, n_filters=128, kernel_size=(3, 3), pad='same', act='relu', name='block2_', layers=2)
  out = ConvBlock(out, n_filters=256, kernel_size=(3, 3), pad='same', act='relu', name='block3_', layers=3)
  out = ConvBlock(out, n_filters=512, kernel_size=(3, 3), pad='same', act='relu', name='block4_', layers=3)
  out = ConvBlock(out, n_filters=512, kernel_size=(3, 3), pad='same', act='relu', name='block5_', layers=3)
  out = Flatten()(out)
  out = Dense(4096, activation='relu', name='fc1')(out)
  out = Dense(4096, activation='relu', name='fc2')(out)
  out = Dense(1000, activation='softmax')(out)
  model = Model(input_img, out)
  model.summary()
  return model

In [4]:
model = VGG16()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_Img (InputLayer)       (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [5]:
from keras.applications.vgg16 import VGG16

pretrained = VGG16(weights=None, include_top=True)
pretrained.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary 


def conv_layer(inp, out, k_size, padding):
  layer = nn.Sequential(
      nn.Conv2d(inp, out, kernel_size=k_size, padding=padding),
      nn.ReLU()
  )
  return layer

def conv_block(in_list, out_list, k_list, p_list):
  layers = [conv_layer(in_list[i], out_list[i], k_list[i], p_list[i]) for i in range(len(in_list))]
  layers += [nn.MaxPool2d(kernel_size=(2, 2))]
  return nn.Sequential(*layers)


def con_fc(inp, out):
  return nn.Sequential(
      nn.Linear(inp, out),
      nn.ReLU()
  )

class VGG_16(nn.Module):
  def __init__(self):
    super(VGG_16, self).__init__()
    self.cb1 = conv_block([3, 64], [64, 64], [3, 3], [1, 1])
    self.cb2 = conv_block([64, 128], [128, 128], [3, 3], [1, 1])
    self.cb3 = conv_block([128, 256, 256], [256, 256, 256], [3, 3, 3], [1, 1, 1])
    self.cb4 = conv_block([256, 512, 512], [512, 512, 512], [3, 3, 3], [1, 1, 1])
    self.cb5 = conv_block([512, 512, 512], [512, 512, 512], [3, 3, 3], [1, 1, 1])


    self.fc1 = con_fc(7*7*512, 4096)
    self.fc2 = con_fc(4096, 4096)
    self.fc3 = nn.Linear(4096, 1000)

  def forward(self, x):
    out = self.cb1(x)
    out = self.cb2(out)
    out = self.cb3(out)
    out = self.cb4(out)
    features = self.cb5(out)
    out = features.view(out.size(0), -1) # Reshaping (m, 7, 7, 512) -> (m, 7 * 7 * 512)
    out = self.fc1(out)
    out = self.fc2(out)
    out = self.fc3(out)
    return out



In [15]:
vgg_torch = VGG_16()

In [16]:
summary(vgg_torch, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [21]:
torch.save(vgg_torch.state_dict(), '/content/model.pkl')

In [27]:
lol = torch.load('/content/model.pkl')
vgg_torch.load_state_dict(lol)

<All keys matched successfully>

In [28]:
summary(vgg_torch, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,